In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from rcrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_rcrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(31943, 1035)

In [6]:
docs_bow = [instance.bow for instance in instances_train]
docs_raw = [[[bow_index]*int(doc_bow[bow_index]) for bow_index in np.where(doc_bow > 0)[0]] for doc_bow in docs_bow]
docs_words = [[idx for idxs in doc for idx in idxs] for doc in docs_raw]
np.sum([len(doc_words) for doc_words in docs_words])

568401

# run

## initialize log

In [7]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [8]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=1, config=config)
train_docs = get_docs(instances_train, config, topic_root=topic_root)
dev_docs = get_docs(instances_dev, config, topic_root=topic_root)
test_docs = get_docs(instances_test, config, topic_root=topic_root)
init(train_docs, dev_docs, test_docs, topic_root)

## run

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
#     ppl_train = get_perplexity(train_docs, topic_root)
    ppl_train = 0.
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2] if 2 in depth_spec else 0, '%.2f'%depth_spec[3] if 3 in depth_spec else 0, \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series
    display(log_df)
    get_freq_tokens_rcrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   156   0  0      0    485   478  0.05  0.41  0.46  0.63  0.59
1   214   1  0      0    442   426  0.05  0.42  0.47  0.57  0.54
2   211   2  0      0    427   410  0.05  0.41  0.44  0.61  0.48
3   200   3  0      0    407   397  0.05  0.38  0.43  0.66  0.43
4   196   4  0      0    394   387  0.05  0.36  0.48  0.61  0.42
5   200   5  0      0    390   382  0.05  0.35  0.42  0.64  0.39
6   199   6  0      0    384   375  0.05  0.33  0.46  0.60  0.40
7   209   7  0      0    375   367  0.05  0.32  0.47  0.55  0.40
8   213   8  0      0    373   364  0.05  0.31  0.47  0.60  0.38
9   216   9  0      0    372   361  0.05  0.37  0.46  0.59  0.40
10  238  10  0      0    369   359  0.05  0.35  0.48  0.71  0.42
11  231  11  0      0    368   359  0.05  0.38  0.48  0.69  0.45
12  240  12  0      0    368   357  0.05  0.36  0.47  0.77  0.43
13  250  13  0      0    365   358  0.05  0.35  0.47  0.65  0.39
14  234  14  0      0    363   357  0.05  0.35  0.48  0.65  0.38
15  227  15  0      0    361   355  0.05  0.35  0.49  0.58  0.40
16  235  16  0      0    361   355  0.05  0.35  0.50  0.57  0.40
17  229  17  0      0    362   355  0.05  0.36  0.48  0.66  0.36
18  233  18  0      0    360   355  0.05  0.36  0.49  0.67  0.36
19  239  19  0      0    359   355  0.05  0.38  0.49  0.59  0.41
20  239  20  0      0    360   355  0.05  0.38  0.51  0.68  0.45
21  247  21  0      0    357   354  0.05  0.36  0.51  0.77  0.39
22  240  22  0      0    357   352  0.05  0.35  0.51  0.70  0.42
23  256  23  0      0    356   350  0.05  0.35  0.51  0.69  0.41
24  249  24  0      0    354   349  0.05  0.38  0.52  0.54  0.43
25  260  25  0      0    355   349  0.05  0.38  0.52  0.61  0.43
26  250  26  0      0    355   349  0.05  0.37  0.52  0.51  0.43
27  250  27  0      0    354   347  0.05  0.37  0.53  0.59  0.41
28  246  28  0      0    353   347  0.05  0.37  0.54  0.65  0.38
29  233  29  0      0    354   347  0.05  0.38  0.55  0.89  0.38

   0 : ['0-1', '0-2', '0-3', '0-4', '0-5', '0-6', '0-7', '0-8', '0-10', '0-11', '0-14', '0-15', '0-16'] 545 9353.0 ['!', 'love', 'cover', 'nice', 'sleeve', ';', 'carry', '&', 'bought', 'price']
     0-1 : ['0-1-1', '0-1-2'] 4873 65530.0 ['cover', 'bottom', '!', 'keyboard', 'color', 'top', 'hard', 'nice', 'pro', 'love']
       0-1-1 : [] 876 11970.0 ['bottom', 'top', 'cover', 'months', 'corners', 'speck', 'plastic', 'part', 'piece', 'back']
       0-1-2 : [] 830 11269.0 ['cover', 'keyboard', 'screen', 'protector', '!', 'hard', 'color', 'keys', 'love', 'perfectly']
     0-2 : [] 7466 98904.0 ['!', 'carry', 'room', 'pockets', 'back', 'love', 'work', 'perfect', 'bought', 'quality']
     0-3 : [] 5465 74083.0 ['pocket', 'room', '!', 'power', 'mouse', 'sleeve', 'carry', 'charger', 'perfect', 'ipad']
     0-4 : [] 5437 60311.0 ['!', 'love', 'color', 'cover', 'perfectly', 'price', 'recommend', 'perfect', 'pro', 'bought']
     0-5 : [] 3781 49042.0 ['zipper', 'months', 'quality', 'strap', 'hand